In [1]:
def shift_char(c, add_key):
    return chr((ord(c) - ord('A') + add_key) % 26 + ord('A'))

In [2]:
def encrypt_vigenere(plaintext, key):
    # Both should be in capital 
    encrypted_text = ""
    key_length = len(key)

    for i, char in enumerate(plaintext):
        shift = ord(key[i % key_length]) - ord('A')
        encrypted_text += shift_char(char, shift)
    
    return encrypted_text

def decrypt_vigenere(ciphertext, key):
    plaintext = ""
    key_length = len(key)
    for i, char in enumerate(ciphertext):
        shift = ord(key[i % key_length]) - ord('A')
        plaintext += shift_char(char, -shift)
        
    return plaintext

In [3]:
# Frequency of English letters
english_freq = {
    'A': 0.08167, 'B': 0.01492, 'C': 0.02782, 'D': 0.04253, 'E': 0.12702,
    'F': 0.02228, 'G': 0.02015, 'H': 0.06094, 'I': 0.06966, 'J': 0.00153,
    'K': 0.00772, 'L': 0.04025, 'M': 0.02406, 'N': 0.06749, 'O': 0.07507,
    'P': 0.01929, 'Q': 0.00095, 'R': 0.05987, 'S': 0.06327, 'T': 0.09056,
    'U': 0.02758, 'V': 0.00978, 'W': 0.02360, 'X': 0.00150, 'Y': 0.01974, 
    'Z': 0.00074
}

In [4]:
import re
def preprocess(text):
    text = re.sub('[^A-Z]', '', text.upper())  # Remove non-alphabet characters and convert to uppercase
    return text

In [5]:
from collections import Counter

def kasiski_examination(ciphertext):
    # Search for repeated sequences of length 3 or more
    repeated_sequences = {}
    for i in range(len(ciphertext) - 2):
        seq = ciphertext[i:i + 3]
        for j in range(i + 3, len(ciphertext) - 2):
            if ciphertext[j:j + 3] == seq:
                if seq not in repeated_sequences:
                    repeated_sequences[seq] = []
                repeated_sequences[seq].append(j - i)
    
    distances = []
    for seq in repeated_sequences:
        distances.extend(repeated_sequences[seq])
    
    key_lengths = Counter()
    for distance in distances:
        for factor in range(2, 21):
            if distance % factor == 0:
                key_lengths[factor] += 1
    
    print("Estimated Key Length :", key_lengths.most_common(1)[0][0])
    return key_lengths.most_common(1)[0][0]

In [6]:
def decrypt_ceaser_cipher(ciphertext):
    text_freq = Counter(ciphertext)
    text_len = len(ciphertext)
    
    chi_squared = {}
    for shift in range(26):
        chi2 = 0
        for char, freq in english_freq.items():
            observed = text_freq[shift_char(char, shift)] / text_len
            chi2 += ((observed - freq) ** 2) / freq
        chi_squared[shift] = chi2
    
    return min(chi_squared, key=chi_squared.get)

In [7]:
# Main decryption function
def decrypt_vigenere_cipher(ciphertext, key_length):
    ciphertext = preprocess(ciphertext)

    key = ""
    for i in range(key_length):
        segment = ciphertext[i::key_length]
        key+=chr(decrypt_ceaser_cipher(segment) + ord('A'))
    
    key = ''.join(key)
    print(f"Estimated key: {key}")

    plaintext = decrypt_vigenere(ciphertext, key)
    return plaintext

In [8]:
plaintext=""" 
Our friendship started when my best friend came in as a new admission to our class. Both of us were hesitant to talk to each other at first, but gradually we developed a bond. I remember the first time my best friend tried to talk to me; I rolled my eyes because I thought there was no use and we wouldn’t hit it off. However, to my surprise, we became best friends by the end of the session year.
We learned so many things about each other and found out that our taste in music and fashion was so similar. Since then, there was no stopping us. We spent all our time together and our friendship became the talk of the class. We used to help each other out in studies and visited each other’s homes as well. We made sure to have lunch together on Sundays. We also used to watch movies and cartoons together.
On our summer break, we even went to summer camp together and made a lot of memories. Once during the summer holidays, she also accompanied me to my maternal grandparents’ home. We had a fabulous time there. Moreover, we even invented our own handshake which only both of us knew. Through this bond, I learned that family doesn’t end with blood because my best friend was no less than my family. Friendship is one relationship that you choose, unlike all other relationships.
"""

ciphertext = encrypt_vigenere(preprocess(plaintext), "MALAY")

key_len = kasiski_examination(ciphertext)
decrypt_vigenere_cipher(ciphertext, key_len)

Estimated Key Length : 5
Estimated key: MALAY


'OURFRIENDSHIPSTARTEDWHENMYBESTFRIENDCAMEINASANEWADMISSIONTOOURCLASSBOTHOFUSWEREHESITANTTOTALKTOEACHOTHERATFIRSTBUTGRADUALLYWEDEVELOPEDABONDIREMEMBERTHEFIRSTTIMEMYBESTFRIENDTRIEDTOTALKTOMEIROLLEDMYEYESBECAUSEITHOUGHTTHEREWASNOUSEANDWEWOULDNTHITITOFFHOWEVERTOMYSURPRISEWEBECAMEBESTFRIENDSBYTHEENDOFTHESESSIONYEARWELEARNEDSOMANYTHINGSABOUTEACHOTHERANDFOUNDOUTTHATOURTASTEINMUSICANDFASHIONWASSOSIMILARSINCETHENTHEREWASNOSTOPPINGUSWESPENTALLOURTIMETOGETHERANDOURFRIENDSHIPBECAMETHETALKOFTHECLASSWEUSEDTOHELPEACHOTHEROUTINSTUDIESANDVISITEDEACHOTHERSHOMESASWELLWEMADESURETOHAVELUNCHTOGETHERONSUNDAYSWEALSOUSEDTOWATCHMOVIESANDCARTOONSTOGETHERONOURSUMMERBREAKWEEVENWENTTOSUMMERCAMPTOGETHERANDMADEALOTOFMEMORIESONCEDURINGTHESUMMERHOLIDAYSSHEALSOACCOMPANIEDMETOMYMATERNALGRANDPARENTSHOMEWEHADAFABULOUSTIMETHEREMOREOVERWEEVENINVENTEDOUROWNHANDSHAKEWHICHONLYBOTHOFUSKNEWTHROUGHTHISBONDILEARNEDTHATFAMILYDOESNTENDWITHBLOODBECAUSEMYBESTFRIENDWASNOLESSTHANMYFAMILYFRIENDSHIPISONERELATIONSHIPTHATYOUCHOOSEUNLIKEALLOT